# Machine Learning(Pandas)

## Pandas
Python中的Pandas库通常用来做数据的读取、分析和预处理，能在数据上做各种变换
### 数据类型
![jupyter](./classes.jfif)

### 新建对象
* Series  
  可以传入一个list对象来新建Series（创建时会默认创建一列索引index，位于最左侧）  
* DataFrame  
  1. 可以传入一个numpy数组，并指定index和列名  
  2. 可以传入一个dict对象

In [5]:
import numpy as np
import pandas as pd
s = pd.Series([1, 3, 4, np.nan, 7]) # 新建Series
print(s)
print()
s = pd.Series([1, 3, 4, np.nan, 7], index=[1, 1, 2, 'a', 4])
print(s)

0    1.0
1    3.0
2    4.0
3    NaN
4    7.0
dtype: float64

1    1.0
1    3.0
2    4.0
a    NaN
4    7.0
dtype: float64


In [9]:
dates = pd.date_range('20190101', periods=6) # 传入列表
print(dates)
print()
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
print(dates)
print()
df2 = pd.DataFrame({'A':1., # 传入字典
                    'B':pd.Timestamp('20190101'),
                    'C':pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D':np.array([3]*4, dtype='int32'),
                    'E':pd.Categorical(["test", "tain", "test", "train"]),
                    'F':'foo'})
print(df2)
print()

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06'],
              dtype='datetime64[ns]', freq='D')

     A          B    C  D      E    F
0  1.0 2019-01-01  1.0  3   test  foo
1  1.0 2019-01-01  1.0  3   tain  foo
2  1.0 2019-01-01  1.0  3   test  foo
3  1.0 2019-01-01  1.0  3  train  foo



In [11]:
df = pd.DataFrame({'A': 132, 'B': 456, 'C': [4121, 45]}) # 广播机制，将A、B两列复制广播
print(df)

     A    B     C
0  132  456  4121
1  132  456    45


### 查看数据
* 查看开头/结尾n行：df.head(n), df.tail(n)
* 查看index/列名：df.index, df.columns
* 查看统计结果：df.describe()
* 转置：df.T
* 按坐标轴排序（0为沿行排序，1为沿列排序）+ 是否降序：df.sort_index(axis=0, ascending=True)

### 筛选数据
* 获取某列：df['A']
* 获取某行：df[0:3]（可切片）
* 通过标签选择：（赋值同理）
    * 选择某行：df.loc[dates[0]]（dates为df的初始数据）
    * 选择指定行列：df.loc['20190102':'20190105', ('A', 'C')]（前一半是行，后一半是列）
    * 选择某值：df.loc['20190102', 'A']，df.at[dates[1], 'A']（前者可以用字符串，后者必须用原来的数据行）
* 通过位置选择：
    * 选择某行：df.iloc[3]（第三行）
    * 选择指定行列：df.iloc[3:7, 0:5]（3\~6行 $\times$ 0\~4列），df.iloc[[1, 2], [3, 4, 5]]
    * 选择某值：df.iloc[1, 1]
* 按条件选择：
    * 按某列的数值选择：df.[df.A > 0]
    * 筛选出合适的数据：df.[df > 0]
    * 使用isin()方法：df['E'].isin(['two', 'four'])（输出一列True,False），df2[df2['E'].isin(['two', 'four'])]（选择出True的那几行）（注意：isin()需严格一致才能匹配）
* 通过NumPy array赋值：df.loc[:, 'D'] = np.array([5] * len(df))（将'D'列设为5）

### 空值处理
pandas默认使用np.nan来表示空值，在统计计算中会直接忽略。
* 修改坐标轴：df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])（选择[0: 4]行并添加'E'列）
* 填充空值：df1.fillna(value=5)（所有np.nan=5）
* 删除空值：df1.dropna(how='any')（整行删除）
* 判断空值：pd.isna(df1)（返回所有值的真假表）

### 运算
* 统计
    注意：所有统计默认不包括空值
    * 平均值：df.mean()（一般按列为单位求，一类数据），df.mean(1)（按行求）
    * 数值移动：s = s.shift(2)（向后移动，新增值用NaN表示）
    * 运算：df.sub(s, axis='index')（广播，以NaN拓展；与NaN相减为NaN）
    * 其他函数见[统计函数整理](https://blog.csdn.net/jiebutianxia/article/details/82216428)
* 应用
    通过apply()方法，对一列数据进行逐一操作
    * 累计求和：df.apply(np.cumsum) = df.cumsum()
    * 自定义方法：df.apply(lambda x: x.max() - x.min())（求极差）
    * 统计元素出现次数：s.value_counts()
    通过map()方法，对一列的每一个数据进行函数操作
    * data[ column_1 ].map(len).map(lambda x: x/100).plot()（链式方法：先化为长度，再除100）
    通过applymap()，给表内所有单元应用一个函数
* String方法：
    调用str的属性方法对每个对象操作
    * 大小写转换：s.str.upper()
    * 分裂：s.str.split(',', expand=True)（分成矩阵，可广播扩展）
    * 其他方法：<https://segmentfault.com/a/1190000018281039?utm_source=tag-newest>

### 合并
* Concat方法：pd.concat([report1, report2], axis=0)（上下接）
* Merge方法：data.merge(other_data, on=[ column_1 ,  column_2 ,  column_3 ])
* Append方法

### 分类目录类型
* 类型转换：df['grade'] = df['raw_grade'].astype('category')（将一列变为category类型）
* 重命名分类：df["grade"].cat.categories = ["very good", "good", "very bad"]（基于category分类）
* 排列：df.sort_values(by="grade")
* 分组：df.groupby("grade").size()
* 生成One hot encode：dummies = pd.get_dummies(df, prefix=None, columns=None)

In [16]:
df = pd.DataFrame({"id": [1, 2, 3, 4, 5, 6],
"raw_grade": ['a', 'b', 'b', 'a', 'a', 'e']})
df['grade'] = df['raw_grade'].astype('category')
df['grade']

0    a
1    b
2    b
3    a
4    a
5    e
Name: grade, dtype: category
Categories (3, object): [a, b, e]

### 导入导出数据
* CSV：pd.read_csv('foo.csv')，df.to_csv('foo.csv')，data = pd.read_csv('my_file.csv' , sep= ,, encoding= latin-1 , nrows=1000, skiprows=[2,5])
* HDF5：pd.read_hdf('foo.h5', 'df')，df.to_hdf('foo.h5', 'df')
* Excel：pd.read_excel('foo.xlsx', 'Sheet1', index_col=None, na_values=['NA'])，df.to_excel('foo.xlsx', sheet_name='Sheet1')

### 绘图
在Jupyter使用时要加上%matplotlib inline
* 折线图：data[column].plot()
* 直方图：data[column].hist()